In [1]:
import os
BASE_FOLDER = os.getcwd()
print(BASE_FOLDER)

c:\Users\Eddy\Desktop\LymphomAug


In [2]:
#from google.colab import drive
#drive.mount(os.path.join(BASE_FOLDER, "drive"))

In [3]:
#import zipfile
#zipf = zipfile.ZipFile("/content/drive/MyDrive/data.zip")
#zipf.extractall(os.path.join(BASE_FOLDER, "data"))
#zipf.close()

In [4]:
#!pip install Augmentor

In [5]:
# !rm -rf /content/data/augmented_images
# !rm -rf /content/data/augmented_labels
# !mkdir /content/data/augmented_images
# !mkdir /content/data/augmented_labels
if not os.path.exists("data/augmented_images"):
    os.makedirs("data/augmented_images")
    os.makedirs("data/augmented_labels")
else:
    os.system("rm data/augmented_images/*")
    os.system("rm data/augmented_labels/*")

In [6]:
import Augmentor
import re
import shutil

def extract_number(s):
    match = re.search(r'\d+', s)
    if match:
        return match.group()
    else:
        return None

p = Augmentor.Pipeline(os.path.join(BASE_FOLDER, "data", "images"), os.path.join(BASE_FOLDER, "data", "augmented_images"))
p.rotate_random_90(probability=0.5)
#p.zoom(probability=0.5, min_factor=1.1, max_factor=1.6)
p.flip_random(probability=0.5)
p.random_contrast(probability=0.5, min_factor=0.8, max_factor=1.2)  # Adjust contrast randomly
p.random_brightness(probability=0.5, min_factor=0.8, max_factor=1.2)  # Adjust brightness randomly
p.random_distortion(probability=0.5, grid_width=4, grid_height=4, magnitude=8)  # Apply random distortion
p.random_color(probability=0.5, min_factor=0.8, max_factor=1.2)  # Randomly change color balance

p.sample(3000)


Initialised with 374 image(s) found.
Output directory set to c:\Users\Eddy\Desktop\LymphomAug\data\augmented_images.

Processing <PIL.Image.Image image mode=RGB size=417x312 at 0x19D64CE69C0>: 100%|██████████| 3000/3000 [00:13<00:00, 224.93 Samples/s]                


In [7]:
start = 374
for filename in os.listdir(os.path.join(BASE_FOLDER, "data", "augmented_images")):
  n = extract_number(filename)
  os.rename(os.path.join(BASE_FOLDER, "data", "augmented_images", filename), os.path.join(BASE_FOLDER, "data", "augmented_images", f"{start}.png"))
  shutil.copyfile(os.path.join(BASE_FOLDER, "data", "labels", f"{n}.txt"), os.path.join(BASE_FOLDER, "data", "augmented_labels", f"{start}.txt"))
  start += 1

In [8]:
#shutil.make_archive("data", 'zip', os.path.join(BASE_FOLDER, "data"))
#shutil.move(os.path.join(BASE_FOLDER, "data.zip"), os.path.join(BASE_FOLDER, "drive", "MyDrive", "data.zip"))